In [1]:
push!(LOAD_PATH,"../../package/QuantumCircuits_demo/src","../../package/VQC_demo_cuda/src")
using VQC
using QuantumCircuits, QuantumCircuits.Gates
using DelimitedFiles
using Random
using Flux.Optimise:ADAM,update!
using Statistics
using StatsBase
using Zygote
using Zygote: @adjoint
using Base: @kwdef
using Plots
using CUDA
using JLD2
device!(1)

CuDevice(1): NVIDIA TITAN V

In [2]:
function build_circuit_xz(N,nlayer)
    circuit = QCircuit()
    for k in 1:nlayer
        for j=1:N
            push!(circuit, RxGate(j, rand(), isparas=true))
            push!(circuit, RzGate(j, rand(), isparas=true))
            push!(circuit, RxGate(j, rand(), isparas=true))
        end 
        for j=1:N-1#div(N,2)
            push!(circuit,CNOTGate(j,j+1))
            #push!(circuit,CNOTGate(N+1-j,N-j) )
        end
    end
    push!(circuit, RxGate(mq[1], rand(), isparas=true))
    push!(circuit, RzGate(mq[1], rand(), isparas=true))
    push!(circuit, RxGate(mq[1], rand(), isparas=true))
    return circuit
end


function loss_util(circuit,data_batch,label_batch,w)
    loss_value = 0.
    out=circuit*data_batch
    p01=real.(expectation(B[1],out))
    L=length(p01)
    p1=p01
    p2=-p01.+1
    loss_value -= dot(label_batch[1,:].*w,log.(p1))
    loss_value -= dot(label_batch[2,:].*w,log.(p2))
    return loss_value/L
end


function fprediction_util(circuit::QCircuit,data_set)#输出预测向量.
    L=nitems(data_set)
    preds=zeros(2,L)
    x=circuit*data_set
    p=real(expectation(B[1],x)) #测量第一个比特0的概率
    preds[1,:]=p
    preds[2,:]=-p.+1
    return preds
end

function fprediction_util(circuits::Vector{QCircuit},alphas::Vector,data_set)#输出预测向量.
    L=nitems(data_set)
    preds=zeros(2,L)
    for i in 1:length(circuits)
        x=circuits[i]*data_set
        p=real(expectation(B[1],x)) #测量第一个比特0的概率
        preds[1,:].+=(p.*alphas[i])
        preds[2,:].+=((-p.+1).*alphas[i])
    end
    return preds
end

function fprediction(circuits::Vector{QCircuit},alphas::Vector,data_set)#输出预测向量.
    L=nitems(data_set)
    preds=fprediction_util(circuits,alphas,data_set)./length(circuits)
    for i in 1:L
        preds[1,i]>0.5 && (preds[:,i]=[1.,0.])
        preds[1,i]<0.5 && (preds[:,i]=[0.,1.])
    end
    return preds
end
function fresult(labels::Matrix,prediction::Matrix)#判断分类的对错，分类正确：+；分类错误：-
    L=size(labels,2)
    result=zeros(L)
    for i in 1:L
        if labels[:,i]!=prediction[:,i]
            result[i]=1
        end
    end
    return result
end

fresult (generic function with 1 method)

In [3]:
h1_vals = [0.1000, 0.2556, 0.4111, 0.5667, 0.7222, 0.8778, 1.0333, 1.1889, 1.3444, 1.5000]
anti_ferro_mag_boundary = [-1.004, -1.0009, -1.024, -1.049, -1.079, -1.109, -1.154,  -1.225, -1.285, -1.35]
para_mag_boundary = [0.8439, 0.6636, 0.5033, 0.3631, 0.2229, 0.09766, -0.02755, -0.1377, -0.2479, -0.3531];

In [4]:
@kwdef mutable struct Args
    η::Float64 = 5e-3       ## learning rate
    batchsize::Int = 40    ## batch size
    epochs::Int = 500      ## number of epochs
end
global args = Args(;)
#分类类别
global const K=2
global N=15
global nlayer=2

global mq=[N] #measure_qubits
#测量算子
UP=[1 0; 0 0]
DOWN=[0 0; 0 1]
const B=[QubitsTerm(mq[1]=>UP),QubitsTerm(mq[1]=>"X"),QubitsTerm(mq[1]=>DOWN)]
Samples=400
interval=0.01
datas = readdlm("SPT_train_set_$(N)_$(Samples)_$(interval).csv",',', Float64)
labels = readdlm("SPT_train_label_$(N)_$(Samples)_$(interval).csv",',')
state=CuStateVectorBatch(CuArray(datas),N,Samples)
C=10

10

In [5]:
circuits = Vector{QCircuit}()
alphas = Float64[]
global W=ones(Samples)./Samples
for c in 1:C
    circuit = build_circuit_xz(N,nlayer)
    opt=ADAM(args.η)
    params = parameters(circuit)
    for iter = 1:args.epochs
        grad = gradient(x->loss_util(x,state,labels,W),circuit)[1]
        update!(opt, params,grad)
        reset_parameters!(circuit, params)
    end
    #第c个弱分类其训练完成
    pred=fprediction([circuit],[1.],state)
    result=fresult(labels,pred)
    ferror=result'*W
    if ferror == 0
        alpha = 10
        push!(alphas,alpha)
        push!(circuits,circuit)
        break
    else
        alpha=log((1-ferror)/ferror)
    end
    println("error rate:$(ferror)")

    #更新权重
    W.=W.*exp.(alpha*result)
    W./=sum(W)
    push!(alphas,alpha)
    push!(circuits,circuit)

    #测试当前线路集性能
    x1=fprediction(circuits,alphas,state)
    re1=dot(x1,labels)
    println("pre_acc_train:$(sum(re1)/Samples)")
    if 1-sum(re1)/Samples<0.05
        break
    end
end

error rate:0.21000000000000002


pre_acc_train:0.735


error rate:0.39323387582881264


pre_acc_train:0.89


error rate:0.1890536605222249


pre_acc_train:0.84


In [ ]:
test_set = readdlm("SPT_test_set_$(N).csv",',', Float64)

In [ ]:
res_pix=zeros(64,64)
res = zeros(64,64)
for i in 1: 64
    test_dm = CuStateVectorBatch(CuArray(test_set[:,(i-1)*64+1:i*64]),N,64)
    re=fprediction(circuits,alphas,test_dm);
    res[:,i]=re[1,:]
end

In [ ]:
heatmap(LinRange(0,1.6,64),LinRange(-1.6,1.6,64),res,c=cgrad([:yellow,:green]))
plot!(h1_vals,[anti_ferro_mag_boundary para_mag_boundary],markershapes=:diamond,markersize=5,lw=3,linestyle=:dash,lc=["red" "blue"],mc=:orange,fc="white",legend=false,title="AdaBoost_$(Samples)")
annotate!([0.75 0.75 0.75] ,[1 -1.5 -0.5],["Paramagnetic","Antiferromagnetic", "SPT"])

In [ ]:
save("AdaBoost_$(Samples)_$(N)q_$(nlayer)_$(interval)_3.jld2","circuits",circuits,"res",res)